### Tucker decomposition (3D)

Let $A = A(i, j, k) \in \mathbb{R}^{n_1 \times n_2 \times n_3}$ be 3-dimensional tensor, $A(i, jk) \in \mathbb{R}^{n_1 \times n_2 n_3}$ - its slice matrix along $i$-axis.

Tucker decomposition algorithm:
1. $A(i, jk) = \{\operatorname{SVD}\} = U_1 \Sigma_1 \tilde{V}$, where $U_1 \in \mathbb{R}^{n_1 \times r_1}$, $\Sigma_1 \in \mathbb{R}^{r_1 \times r_1}$, $\tilde{V} \in \mathbb{R}^{r_1 \times n_2 n_3}$.
2. $\operatorname{reshape}(\tilde{V}) = \hat{V} \in \mathbb{R}^{r_1 n_2 \times n_3}$.
3. $\hat{V} = \{\operatorname{SVD}\} = \tilde{U} \Sigma_2 V_2$, where $U_2 \in \mathbb{R}^{r_1 n_2 \times r_2}$, $\Sigma_2 \in \mathbb{R}^{r_2 \times r_2}$, $V_2 \in \mathbb{R}^{r_2 \times n_3}$.
4. $\operatorname{reshape~\&~transpose}(\tilde{U}) = \hat{U} \in \mathbb{R}^{r_1 r_2 \times n_2}$.
5. $\hat{V} = \{\operatorname{SVD}\} = U_3 \Sigma_3 V_3$, where $U_3 \in \mathbb{R}^{r_1 r_2 \times r_3}$, $\Sigma_3 \in \mathbb{R}^{r_3 \times r_3}$, $V_3 \in \mathbb{R}^{r_3 \times n_2}$.
6. Factors of 3-dimensional Tucker decomposition: $U = (U_1 \Sigma_1)^T \in \mathbb{R}^{r_1 \times n_1}$, $V = \Sigma_2 V_2 \in \mathbb{R}^{r_3 \times n_2}$, $W = \Sigma_3 V_3 \in \mathbb{R}^{r_2 \times n_3}$, $G = \operatorname{reshape}(U_3) \in \mathbb{R}^{r_1 \times r_2 \times r_3}$

In [1]:
import numpy as np

In [2]:
N = 128
i, j, k = np.meshgrid(np.arange(N), np.arange(N), np.arange(N), indexing='ij')
A = (i + j + k)**2
A = A.astype(float)
Ai_jk = A.reshape(N, N*N)

In [3]:
U_1, s_1, V_1 = np.linalg.svd(Ai_jk, full_matrices=False)
s_1 = s_1[s_1 > 1e-6]
r_1 = len(s_1)
U_1 = U_1[:, :r_1]; V_1 = V_1[:r_1, :]
U = U_1 @ np.diag(s_1)

In [4]:
V_1 = V_1.reshape(r_1, N, N).reshape(r_1*N, N)

U_2, s_2, V_2 = np.linalg.svd(V_1, full_matrices=False)
s_2 = s_2[s_2 > 1e-6]
r_2 = len(s_2)
U_2 = U_2[:, :r_2]; V_2 = V_2[:r_2, :]
W = np.diag(s_2) @ V_2

In [5]:
U_2 = U_2.reshape(r_1, N, r_2).transpose([0, 2, 1]).reshape(r_1*r_2, N)

U_3, s_3, V_3 = np.linalg.svd(U_2, full_matrices=False)
s_3 = s_3[s_3 > 1e-6]
r_3 = len(s_3)
U_3 = U_3[:, :r_3]; V_3 = V_3[:r_3, :]
V = np.diag(s_3) @ V_3
G = U_3.reshape(r_1, r_2, r_3)

In [6]:
print(U.shape, G.shape, V.shape, W.shape)

(128, 3) (3, 3, 3) (3, 128) (3, 128)


In [7]:
reconstruct_tensor = (U @ ((G.reshape(r_1*r_2, r_3) @ V).reshape(r_1, r_2, N).transpose([0, 2, 1]).reshape(r_1*N, r_2) @ W).reshape(r_1, N, N).reshape(r_1, N*N)).reshape(N, N, N)

In [8]:
np.linalg.norm((A - reconstruct_tensor).flatten())

1.396928648862498e-06